# 87. 受容野と層の深さ：階層的抽象化の数理

## 学習目標

このノートブックでは、以下を学びます：

1. **層を重ねると受容野がどう成長するか**の数学的理解
2. **受容野サイズの漸化式**の導出と証明
3. **階層的特徴抽象化**と受容野の関係
4. **深いネットワーク**がなぜ複雑なパターンを捉えられるか

## 目次

1. [復習：単一層の受容野](#section1)
2. [2層での受容野成長](#section2)
3. [漸化式の導出](#section3)
4. [累積ストライドの役割](#section4)
5. [階層的特徴抽象化](#section5)
6. [実際のネットワークでの計算](#section6)
7. [まとめ](#summary)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib.colors import LinearSegmentedColormap
import japanize_matplotlib

plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

<a id="section1"></a>
## 1. 復習：単一層の受容野

前のノートブックで学んだように、**受容野（Receptive Field, RF）**とは：

> ある出力ニューロンが「見ている」入力領域のサイズ

### 単一畳み込み層の場合

カーネルサイズが $k$ の畳み込み層では：

$$\text{RF} = k$$

例：3×3カーネル → 受容野 = 3×3

In [ ]:
def visualize_single_layer_rf(kernel_size=3):
    """単一層の受容野を可視化"""
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # 入力画像（7x7）
    input_size = 7
    ax = axes[0]
    
    # グリッド描画
    for i in range(input_size + 1):
        ax.axhline(y=i, color='gray', linewidth=0.5)
        ax.axvline(x=i, color='gray', linewidth=0.5)
    
    # 受容野をハイライト（中央に配置）
    rf_start = (input_size - kernel_size) // 2
    rect = Rectangle((rf_start, rf_start), kernel_size, kernel_size,
                      linewidth=3, edgecolor='red', facecolor='lightyellow', alpha=0.7)
    ax.add_patch(rect)
    
    ax.set_xlim(-0.5, input_size + 0.5)
    ax.set_ylim(-0.5, input_size + 0.5)
    ax.set_aspect('equal')
    ax.set_title(f'入力層 ({input_size}×{input_size})', fontsize=14)
    ax.invert_yaxis()
    ax.set_xlabel('受容野 = カーネルサイズ', fontsize=12)
    
    # 出力の1ピクセル
    ax = axes[1]
    output_size = input_size - kernel_size + 1  # valid padding
    
    for i in range(output_size + 1):
        ax.axhline(y=i, color='gray', linewidth=0.5)
        ax.axvline(x=i, color='gray', linewidth=0.5)
    
    # 対応する出力ピクセル
    out_pos = (output_size - 1) // 2
    rect = Rectangle((out_pos, out_pos), 1, 1,
                      linewidth=3, edgecolor='red', facecolor='lightcoral', alpha=0.7)
    ax.add_patch(rect)
    
    ax.set_xlim(-0.5, output_size + 0.5)
    ax.set_ylim(-0.5, output_size + 0.5)
    ax.set_aspect('equal')
    ax.set_title(f'出力層 ({output_size}×{output_size})', fontsize=14)
    ax.invert_yaxis()
    ax.set_xlabel('この1ピクセル', fontsize=12)
    
    # 矢印
    fig.text(0.5, 0.5, '←\n対応', fontsize=16, ha='center', va='center')
    
    plt.suptitle(f'単一層の受容野：{kernel_size}×{kernel_size}カーネル → RF = {kernel_size}×{kernel_size}', 
                 fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

visualize_single_layer_rf(kernel_size=3)

<a id="section2"></a>
## 2. 2層での受容野成長

### 核心的な洞察

2層目の各ニューロンは、**1層目の出力**を見ています。  
1層目の各ニューロンは、**元の入力**の一部を見ています。

したがって、2層目のニューロンが見る入力領域は：

$$\text{1層目が見る領域} + \text{2層目が追加で見る範囲}$$

In [ ]:
def visualize_two_layer_rf():
    """2層の受容野成長を詳細に可視化"""
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    
    k1, k2 = 3, 3  # 両層とも3x3カーネル
    input_size = 9
    
    # Layer 0: 入力
    ax = axes[0]
    for i in range(input_size + 1):
        ax.axhline(y=i, color='gray', linewidth=0.5)
        ax.axvline(x=i, color='gray', linewidth=0.5)
    
    # 2層目の受容野（5x5）
    rf2 = k1 + k2 - 1  # = 5
    rf_start = (input_size - rf2) // 2
    rect = Rectangle((rf_start, rf_start), rf2, rf2,
                      linewidth=3, edgecolor='blue', facecolor='lightblue', alpha=0.5)
    ax.add_patch(rect)
    
    # 1層目の受容野を内側に表示
    for i in range(k2):
        for j in range(k2):
            rect = Rectangle((rf_start + i, rf_start + j), k1, k1,
                              linewidth=1, edgecolor='red', facecolor='none', 
                              linestyle='--', alpha=0.5)
            # ax.add_patch(rect)
    
    ax.set_xlim(-0.5, input_size + 0.5)
    ax.set_ylim(-0.5, input_size + 0.5)
    ax.set_aspect('equal')
    ax.set_title(f'入力 ({input_size}×{input_size})\n2層目のRF = {rf2}×{rf2}', fontsize=14)
    ax.invert_yaxis()
    
    # Layer 1: 1層目の出力
    ax = axes[1]
    l1_size = input_size - k1 + 1  # = 7
    
    for i in range(l1_size + 1):
        ax.axhline(y=i, color='gray', linewidth=0.5)
        ax.axvline(x=i, color='gray', linewidth=0.5)
    
    # 2層目のカーネルが見る範囲（3x3）
    l1_rf_start = (l1_size - k2) // 2
    rect = Rectangle((l1_rf_start, l1_rf_start), k2, k2,
                      linewidth=3, edgecolor='red', facecolor='lightyellow', alpha=0.7)
    ax.add_patch(rect)
    
    ax.set_xlim(-0.5, l1_size + 0.5)
    ax.set_ylim(-0.5, l1_size + 0.5)
    ax.set_aspect('equal')
    ax.set_title(f'1層目出力 ({l1_size}×{l1_size})\n2層目のカーネル = {k2}×{k2}', fontsize=14)
    ax.invert_yaxis()
    
    # Layer 2: 2層目の出力
    ax = axes[2]
    l2_size = l1_size - k2 + 1  # = 5
    
    for i in range(l2_size + 1):
        ax.axhline(y=i, color='gray', linewidth=0.5)
        ax.axvline(x=i, color='gray', linewidth=0.5)
    
    # 対応する出力ピクセル
    out_pos = (l2_size - 1) // 2
    rect = Rectangle((out_pos, out_pos), 1, 1,
                      linewidth=3, edgecolor='red', facecolor='lightcoral', alpha=0.7)
    ax.add_patch(rect)
    
    ax.set_xlim(-0.5, l2_size + 0.5)
    ax.set_ylim(-0.5, l2_size + 0.5)
    ax.set_aspect('equal')
    ax.set_title(f'2層目出力 ({l2_size}×{l2_size})\nこの1ピクセル', fontsize=14)
    ax.invert_yaxis()
    
    # 説明テキスト
    fig.text(0.35, 0.02, f'3×3 + 3×3 = 5×5 RF (NOT 6×6!)', 
             fontsize=14, ha='center', fontweight='bold', color='blue')
    
    plt.suptitle('2層の畳み込みによる受容野の成長', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.subplots_adjust(bottom=0.1)
    plt.show()

visualize_two_layer_rf()

### なぜ 3+3=5 なのか？（6ではない）

直感的な説明：

```
入力:     [1] [2] [3] [4] [5] [6] [7]
          \_____/
           ↓ 3x3カーネル
1層目:       [A] [B] [C] [D] [E]
             \_____/
              ↓ 3x3カーネル  
2層目:          [X] [Y] [Z]
```

- 2層目の`[Y]`は1層目の`[B],[C],[D]`を見る
- `[B]`は入力の`[2],[3],[4]`を見る
- `[C]`は入力の`[3],[4],[5]`を見る
- `[D]`は入力の`[4],[5],[6]`を見る

合計すると、`[Y]`は入力の`[2],[3],[4],[5],[6]`を見る → **5ピクセル**

**重なりがある**ため、単純な足し算にはならない！

In [ ]:
def demonstrate_rf_overlap():
    """受容野の重なりを1Dで可視化"""
    fig, ax = plt.subplots(figsize=(14, 8))
    
    # 入力層（7ピクセル）
    input_labels = ['1', '2', '3', '4', '5', '6', '7']
    y_input = 3
    for i, label in enumerate(input_labels):
        rect = Rectangle((i, y_input), 1, 1, linewidth=1, 
                         edgecolor='black', facecolor='lightgray')
        ax.add_patch(rect)
        ax.text(i + 0.5, y_input + 0.5, label, ha='center', va='center', fontsize=12)
    ax.text(-1, y_input + 0.5, '入力:', ha='right', va='center', fontsize=12, fontweight='bold')
    
    # 1層目（5ピクセル）
    l1_labels = ['A', 'B', 'C', 'D', 'E']
    y_l1 = 2
    for i, label in enumerate(l1_labels):
        rect = Rectangle((i + 1, y_l1), 1, 1, linewidth=1, 
                         edgecolor='black', facecolor='lightyellow')
        ax.add_patch(rect)
        ax.text(i + 1.5, y_l1 + 0.5, label, ha='center', va='center', fontsize=12)
    ax.text(-1, y_l1 + 0.5, '1層目:', ha='right', va='center', fontsize=12, fontweight='bold')
    
    # 2層目（3ピクセル）
    l2_labels = ['X', 'Y', 'Z']
    y_l2 = 1
    for i, label in enumerate(l2_labels):
        rect = Rectangle((i + 2, y_l2), 1, 1, linewidth=1, 
                         edgecolor='black', facecolor='lightcoral')
        ax.add_patch(rect)
        ax.text(i + 2.5, y_l2 + 0.5, label, ha='center', va='center', fontsize=12)
    ax.text(-1, y_l2 + 0.5, '2層目:', ha='right', va='center', fontsize=12, fontweight='bold')
    
    # Yの受容野を強調
    # Yが見る1層目: B, C, D
    for i in [1, 2, 3]:
        rect = Rectangle((i + 1, y_l1), 1, 1, linewidth=3, 
                         edgecolor='red', facecolor='none')
        ax.add_patch(rect)
    
    # 入力での受容野
    rect = Rectangle((1, y_input), 5, 1, linewidth=3, 
                     edgecolor='blue', facecolor='lightblue', alpha=0.3)
    ax.add_patch(rect)
    
    # 接続線
    # Y -> B, C, D
    ax.annotate('', xy=(3.5, y_l2), xytext=(2.5, y_l1 + 1),
                arrowprops=dict(arrowstyle='->', color='red', lw=2))
    ax.annotate('', xy=(3.5, y_l2), xytext=(3.5, y_l1 + 1),
                arrowprops=dict(arrowstyle='->', color='red', lw=2))
    ax.annotate('', xy=(3.5, y_l2), xytext=(4.5, y_l1 + 1),
                arrowprops=dict(arrowstyle='->', color='red', lw=2))
    
    # B, C, D が見る範囲を個別に表示
    y_detail = 4.5
    ax.text(0.5, y_detail, 'B が見る: [2,3,4]', fontsize=11, color='darkred')
    ax.text(2.5, y_detail, 'C が見る: [3,4,5]', fontsize=11, color='darkred')
    ax.text(4.5, y_detail, 'D が見る: [4,5,6]', fontsize=11, color='darkred')
    
    ax.text(3.5, 0.3, 'Y の受容野: [2,3,4,5,6] = 5ピクセル', 
            fontsize=14, ha='center', fontweight='bold', color='blue')
    
    ax.set_xlim(-2, 8)
    ax.set_ylim(-0.5, 5.5)
    ax.set_aspect('equal')
    ax.axis('off')
    ax.set_title('受容野の重なり：3×3 + 3×3 → 5（重複があるため6にならない）', 
                 fontsize=16, fontweight='bold')
    
    plt.tight_layout()
    plt.show()

demonstrate_rf_overlap()

<a id="section3"></a>
## 3. 漸化式の導出

### 一般的な公式

$n$ 層までの受容野サイズを $\text{RF}_n$ とすると：

$$\boxed{\text{RF}_n = \text{RF}_{n-1} + (k_n - 1) \times \prod_{i=1}^{n-1} s_i}$$

ここで：
- $k_n$：$n$ 層目のカーネルサイズ
- $s_i$：$i$ 層目のストライド
- $\prod_{i=1}^{n-1} s_i$：**累積ストライド**（後で詳しく説明）

### ストライド1の場合（単純化）

すべてのストライドが1の場合：

$$\text{RF}_n = \text{RF}_{n-1} + (k_n - 1)$$

すなわち：

$$\text{RF}_n = 1 + \sum_{i=1}^{n} (k_i - 1)$$

In [ ]:
def prove_rf_formula():
    """漸化式を視覚的に証明"""
    print("="*60)
    print("受容野の漸化式の証明（ストライド=1の場合）")
    print("="*60)
    
    print("\n【定義】")
    print("RF_n = n層目の出力1ピクセルが入力画像上で見る領域のサイズ")
    
    print("\n【基底】")
    print("RF_0 = 1 （入力の1ピクセルは自分自身のみ）")
    print("RF_1 = k_1 （1層目のカーネルサイズ）")
    
    print("\n【帰納】")
    print("n層目の出力1ピクセルは、(n-1)層目の k_n ピクセルを見る")
    print("(n-1)層目の各ピクセルは、入力画像の RF_{n-1} ピクセルを見る")
    print("")
    print("隣接するピクセルの受容野は (RF_{n-1} - 1) ピクセル重なる")
    print("したがって：")
    print("  RF_n = RF_{n-1} + (k_n - 1) × (重複しない新しいピクセル数)")
    print("       = RF_{n-1} + (k_n - 1)  [ストライド=1の場合]")
    
    print("\n【計算例：3×3カーネルを4層重ねた場合】")
    rf = 1  # RF_0
    print(f"RF_0 = {rf}")
    
    for n in range(1, 5):
        k = 3
        rf_new = rf + (k - 1)
        print(f"RF_{n} = RF_{n-1} + (k_{n} - 1) = {rf} + ({k} - 1) = {rf_new}")
        rf = rf_new
    
    print(f"\n結論：3×3カーネル4層 → 受容野 = {rf}×{rf}")
    print("\n【一般公式】")
    print("RF_n = 1 + Σ(k_i - 1) for i=1 to n")
    print("     = 1 + n × (k - 1)  [すべて同じカーネルサイズkの場合]")

prove_rf_formula()

In [ ]:
def visualize_rf_growth():
    """受容野の成長を可視化"""
    kernel_sizes = [3, 5, 7]
    max_layers = 10
    
    fig, ax = plt.subplots(figsize=(12, 8))
    
    for k in kernel_sizes:
        layers = range(max_layers + 1)
        rfs = [1 + n * (k - 1) for n in layers]
        ax.plot(layers, rfs, 'o-', label=f'{k}×{k} カーネル', linewidth=2, markersize=8)
        
        # 最終値をアノテート
        ax.annotate(f'RF={rfs[-1]}', xy=(max_layers, rfs[-1]), 
                   xytext=(max_layers + 0.3, rfs[-1]),
                   fontsize=11, va='center')
    
    ax.set_xlabel('層の数', fontsize=14)
    ax.set_ylabel('受容野サイズ', fontsize=14)
    ax.set_title('層の深さと受容野サイズの関係\n（ストライド=1の場合）', fontsize=16, fontweight='bold')
    ax.legend(fontsize=12)
    ax.grid(True, alpha=0.3)
    ax.set_xticks(range(0, max_layers + 1))
    
    # 公式を追加
    ax.text(0.02, 0.98, 'RF = 1 + n × (k - 1)', transform=ax.transAxes,
           fontsize=14, verticalalignment='top', 
           bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    plt.tight_layout()
    plt.show()

visualize_rf_growth()

### 重要な洞察

1. **3×3カーネルを2層** → RF = 5×5 = 1 + 2×2
2. **3×3カーネルを3層** → RF = 7×7 = 1 + 3×2
3. **5×5カーネル1層** → RF = 5×5

つまり、**3×3を2層重ねると5×5と同等の受容野**が得られる！

しかも、パラメータ数は：
- 5×5×1 = 25
- 3×3×2 = 18

**小さいカーネルを重ねる方が効率的**（VGGNetの設計思想）

In [ ]:
def compare_kernel_efficiency():
    """同じ受容野を得るためのカーネル比較"""
    comparisons = [
        {'target_rf': 5, 'configs': [(5, 1), (3, 2)]},
        {'target_rf': 7, 'configs': [(7, 1), (3, 3)]},
        {'target_rf': 9, 'configs': [(9, 1), (5, 2), (3, 4)]},
        {'target_rf': 11, 'configs': [(11, 1), (3, 5)]},
    ]
    
    print("="*70)
    print("同じ受容野を達成するための構成比較")
    print("="*70)
    
    for comp in comparisons:
        target = comp['target_rf']
        print(f"\n目標受容野: {target}×{target}")
        print("-" * 50)
        
        for k, n in comp['configs']:
            rf = 1 + n * (k - 1)
            params = k * k * n
            nonlinearities = n
            print(f"  {k}×{k} × {n}層: RF={rf}, パラメータ={params:3d}, 非線形活性化={nonlinearities}回")

compare_kernel_efficiency()

<a id="section4"></a>
## 4. 累積ストライドの役割

ストライドが1より大きい場合、受容野はより急速に拡大します。

### 累積ストライドとは

$$\text{累積ストライド}_n = \prod_{i=1}^{n} s_i$$

これは「$n$層目の出力で1ピクセル動くと、入力では何ピクセル動くか」を表します。

In [ ]:
def visualize_stride_effect():
    """ストライドが受容野に与える影響を可視化"""
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    
    # ストライド=2の場合
    # 入力
    ax = axes[0]
    input_size = 9
    for i in range(input_size + 1):
        ax.axhline(y=i, color='gray', linewidth=0.5)
        ax.axvline(x=i, color='gray', linewidth=0.5)
    
    # 1層目の受容野（3x3, stride=2）
    # 2層目のRFを表示
    # RF = 1 + (3-1) + (3-1)*2 = 1 + 2 + 4 = 7
    rf_size = 7
    rect = Rectangle((1, 1), rf_size, rf_size,
                      linewidth=3, edgecolor='blue', facecolor='lightblue', alpha=0.5)
    ax.add_patch(rect)
    
    ax.set_xlim(-0.5, input_size + 0.5)
    ax.set_ylim(-0.5, input_size + 0.5)
    ax.set_aspect('equal')
    ax.set_title(f'入力 ({input_size}×{input_size})\n2層目のRF = {rf_size}×{rf_size}', fontsize=14)
    ax.invert_yaxis()
    
    # 1層目（stride=2で3x3）
    ax = axes[1]
    l1_size = (input_size - 3) // 2 + 1  # = 4
    
    for i in range(l1_size + 1):
        ax.axhline(y=i, color='gray', linewidth=0.5)
        ax.axvline(x=i, color='gray', linewidth=0.5)
    
    # 2層目のカーネルが見る範囲
    rect = Rectangle((0, 0), 3, 3,
                      linewidth=3, edgecolor='red', facecolor='lightyellow', alpha=0.7)
    ax.add_patch(rect)
    
    ax.set_xlim(-0.5, l1_size + 0.5)
    ax.set_ylim(-0.5, l1_size + 0.5)
    ax.set_aspect('equal')
    ax.set_title(f'1層目出力 ({l1_size}×{l1_size})\n3×3カーネル', fontsize=14)
    ax.invert_yaxis()
    
    # 2層目
    ax = axes[2]
    l2_size = l1_size - 3 + 1  # = 2
    
    for i in range(l2_size + 1):
        ax.axhline(y=i, color='gray', linewidth=0.5)
        ax.axvline(x=i, color='gray', linewidth=0.5)
    
    rect = Rectangle((0, 0), 1, 1,
                      linewidth=3, edgecolor='red', facecolor='lightcoral', alpha=0.7)
    ax.add_patch(rect)
    
    ax.set_xlim(-0.5, l2_size + 0.5)
    ax.set_ylim(-0.5, l2_size + 0.5)
    ax.set_aspect('equal')
    ax.set_title(f'2層目出力 ({l2_size}×{l2_size})', fontsize=14)
    ax.invert_yaxis()
    
    plt.suptitle('ストライド=2の影響：3×3(s=2) + 3×3(s=1) → RF=7×7', 
                 fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

visualize_stride_effect()

In [ ]:
class ReceptiveFieldCalculator:
    """受容野計算ユーティリティ（ストライド対応版）"""
    
    def __init__(self):
        self.layers = []
        
    def add_conv(self, kernel_size, stride=1, padding=0, name=None):
        """畳み込み層を追加"""
        self.layers.append({
            'type': 'conv',
            'kernel_size': kernel_size,
            'stride': stride,
            'padding': padding,
            'name': name or f'Conv{len(self.layers)+1}'
        })
        return self
    
    def add_pool(self, kernel_size, stride=None, name=None):
        """プーリング層を追加"""
        if stride is None:
            stride = kernel_size
        self.layers.append({
            'type': 'pool',
            'kernel_size': kernel_size,
            'stride': stride,
            'padding': 0,
            'name': name or f'Pool{len(self.layers)+1}'
        })
        return self
    
    def calculate(self):
        """各層の受容野を計算"""
        results = [{'name': 'Input', 'rf': 1, 'cumulative_stride': 1}]
        
        rf = 1
        cumulative_stride = 1
        
        for layer in self.layers:
            k = layer['kernel_size']
            s = layer['stride']
            
            # 受容野の更新
            rf = rf + (k - 1) * cumulative_stride
            
            results.append({
                'name': layer['name'],
                'rf': rf,
                'cumulative_stride': cumulative_stride * s,
                'kernel': k,
                'stride': s
            })
            
            # 累積ストライドの更新
            cumulative_stride *= s
        
        return results
    
    def print_summary(self):
        """結果を表形式で表示"""
        results = self.calculate()
        
        print("\n" + "="*70)
        print(f"{'層名':<15} {'カーネル':>8} {'ストライド':>10} {'累積stride':>12} {'受容野':>8}")
        print("="*70)
        
        for i, r in enumerate(results):
            if i == 0:
                print(f"{r['name']:<15} {'-':>8} {'-':>10} {r['cumulative_stride']:>12} {r['rf']:>8}")
            else:
                print(f"{r['name']:<15} {r['kernel']:>8} {r['stride']:>10} {r['cumulative_stride']:>12} {r['rf']:>8}")
        
        print("="*70)
        print(f"最終受容野: {results[-1]['rf']}×{results[-1]['rf']}")
        
        return results

# 例1: ストライド=1のみ
print("例1: 3×3 カーネル × 4層（すべてstride=1）")
calc = ReceptiveFieldCalculator()
for i in range(4):
    calc.add_conv(3, stride=1, name=f'Conv{i+1}')
calc.print_summary()

# 例2: ストライドあり
print("\n例2: ストライドを含む構成")
calc2 = ReceptiveFieldCalculator()
calc2.add_conv(3, stride=1, name='Conv1')
calc2.add_conv(3, stride=2, name='Conv2(s=2)')
calc2.add_conv(3, stride=1, name='Conv3')
calc2.add_conv(3, stride=2, name='Conv4(s=2)')
calc2.print_summary()

### ストライドの効果

上の例2を分析：

1. **Conv1**: RF = 1 + (3-1)×1 = 3, 累積stride = 1
2. **Conv2(s=2)**: RF = 3 + (3-1)×1 = 5, 累積stride = 2
3. **Conv3**: RF = 5 + (3-1)×2 = 9, 累積stride = 2
4. **Conv4(s=2)**: RF = 9 + (3-1)×2 = 13, 累積stride = 4

**ストライド>1は、それ以降の層の受容野成長を加速させる！**

In [ ]:
def compare_stride_vs_no_stride():
    """ストライドの有無による受容野成長の比較"""
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # ストライドなし（すべてs=1）
    calc1 = ReceptiveFieldCalculator()
    for i in range(8):
        calc1.add_conv(3, stride=1)
    results1 = calc1.calculate()
    rfs1 = [r['rf'] for r in results1]
    
    # 2層ごとにストライド2
    calc2 = ReceptiveFieldCalculator()
    for i in range(8):
        s = 2 if (i + 1) % 2 == 0 else 1
        calc2.add_conv(3, stride=s)
    results2 = calc2.calculate()
    rfs2 = [r['rf'] for r in results2]
    
    layers = range(len(rfs1))
    
    ax.plot(layers, rfs1, 'o-', label='すべてstride=1', linewidth=2, markersize=8)
    ax.plot(layers, rfs2, 's-', label='2層ごとにstride=2', linewidth=2, markersize=8)
    
    ax.set_xlabel('層の数', fontsize=14)
    ax.set_ylabel('受容野サイズ', fontsize=14)
    ax.set_title('ストライドが受容野成長に与える影響', fontsize=16, fontweight='bold')
    ax.legend(fontsize=12)
    ax.grid(True, alpha=0.3)
    
    # 最終値を表示
    ax.annotate(f'RF={rfs1[-1]}', xy=(8, rfs1[-1]), xytext=(8.3, rfs1[-1]),
               fontsize=11, va='center')
    ax.annotate(f'RF={rfs2[-1]}', xy=(8, rfs2[-1]), xytext=(8.3, rfs2[-1]),
               fontsize=11, va='center')
    
    plt.tight_layout()
    plt.show()

compare_stride_vs_no_stride()

<a id="section5"></a>
## 5. 階層的特徴抽象化

### 受容野と認識能力の関係

受容野が大きくなると、ネットワークはより大きなパターンを認識できるようになります。

| 層の深さ | 受容野サイズ | 認識できるパターン |
|---------|------------|------------------|
| 浅い層 | 小さい | エッジ、テクスチャ |
| 中間層 | 中程度 | パーツ、形状 |
| 深い層 | 大きい | オブジェクト全体 |

In [ ]:
def visualize_hierarchical_abstraction():
    """階層的特徴抽象化の概念図"""
    fig, axes = plt.subplots(1, 4, figsize=(18, 5))
    
    # サンプル画像（顔の模式図）
    np.random.seed(42)
    
    # 入力画像
    ax = axes[0]
    img = np.zeros((32, 32))
    # 顔の輪郭（円）
    y, x = np.ogrid[:32, :32]
    center = (16, 16)
    r = 12
    mask = (x - center[0])**2 + (y - center[1])**2 < r**2
    img[mask] = 0.8
    # 目
    img[10:14, 10:14] = 0.2
    img[10:14, 18:22] = 0.2
    # 口
    img[20:22, 12:20] = 0.2
    
    ax.imshow(img, cmap='gray', vmin=0, vmax=1)
    ax.set_title('入力画像\n(32×32)', fontsize=14)
    ax.axis('off')
    
    # 層1: エッジ検出（RF=3）
    ax = axes[1]
    # Sobelでエッジ検出をシミュレート
    from scipy import ndimage
    edges = ndimage.sobel(img)
    ax.imshow(np.abs(edges), cmap='hot')
    rect = Rectangle((0, 0), 3, 3, linewidth=2, edgecolor='blue', facecolor='none')
    ax.add_patch(rect)
    ax.set_title('層1-2: エッジ\nRF=3-5', fontsize=14)
    ax.axis('off')
    
    # 層3-4: パーツ（RF=7-9）
    ax = axes[2]
    # 中間特徴をシミュレート（ぼかし＋閾値）
    blurred = ndimage.gaussian_filter(img, sigma=2)
    ax.imshow(blurred, cmap='viridis')
    rect = Rectangle((0, 0), 9, 9, linewidth=2, edgecolor='blue', facecolor='none')
    ax.add_patch(rect)
    ax.set_title('層3-4: パーツ\nRF=7-9', fontsize=14)
    ax.axis('off')
    
    # 深い層: オブジェクト（RF=15+）
    ax = axes[3]
    # 全体的な特徴（大きく平均化）
    pooled = ndimage.zoom(ndimage.zoom(img, 0.25), 4)
    ax.imshow(pooled, cmap='coolwarm')
    rect = Rectangle((0, 0), 17, 17, linewidth=2, edgecolor='blue', facecolor='none')
    ax.add_patch(rect)
    ax.set_title('層5+: オブジェクト\nRF=17+', fontsize=14)
    ax.axis('off')
    
    plt.suptitle('受容野と階層的特徴抽象化', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

visualize_hierarchical_abstraction()

### なぜ深いネットワークが必要なのか？

1. **大きな受容野が必要**：オブジェクト全体を見るため
2. **小さいカーネルの方が効率的**：パラメータ数が少ない
3. **非線形性の積み重ね**：複雑な関数を近似できる

例：224×224の画像で顔全体（約100×100）を認識するには：
- 100×100の受容野が必要
- 3×3カーネルで達成するには約50層必要（理論的には）
- ストライドやプーリングを使えば、より少ない層で達成可能

<a id="section6"></a>
## 6. 実際のネットワークでの計算

実際の有名なネットワークの受容野を計算してみましょう。

In [ ]:
def calculate_vgg_rf():
    """VGG16の受容野計算"""
    print("\n" + "="*70)
    print("VGG16の受容野計算")
    print("="*70)
    
    calc = ReceptiveFieldCalculator()
    
    # Block 1
    calc.add_conv(3, stride=1, name='conv1_1')
    calc.add_conv(3, stride=1, name='conv1_2')
    calc.add_pool(2, stride=2, name='pool1')
    
    # Block 2
    calc.add_conv(3, stride=1, name='conv2_1')
    calc.add_conv(3, stride=1, name='conv2_2')
    calc.add_pool(2, stride=2, name='pool2')
    
    # Block 3
    calc.add_conv(3, stride=1, name='conv3_1')
    calc.add_conv(3, stride=1, name='conv3_2')
    calc.add_conv(3, stride=1, name='conv3_3')
    calc.add_pool(2, stride=2, name='pool3')
    
    # Block 4
    calc.add_conv(3, stride=1, name='conv4_1')
    calc.add_conv(3, stride=1, name='conv4_2')
    calc.add_conv(3, stride=1, name='conv4_3')
    calc.add_pool(2, stride=2, name='pool4')
    
    # Block 5
    calc.add_conv(3, stride=1, name='conv5_1')
    calc.add_conv(3, stride=1, name='conv5_2')
    calc.add_conv(3, stride=1, name='conv5_3')
    calc.add_pool(2, stride=2, name='pool5')
    
    return calc.print_summary()

vgg_results = calculate_vgg_rf()

In [ ]:
def calculate_resnet_rf():
    """ResNet-18の受容野計算（簡略版）"""
    print("\n" + "="*70)
    print("ResNet-18の受容野計算（簡略版）")
    print("="*70)
    
    calc = ReceptiveFieldCalculator()
    
    # Stem
    calc.add_conv(7, stride=2, name='conv1')
    calc.add_pool(3, stride=2, name='maxpool')
    
    # Layer 1 (no downsampling)
    calc.add_conv(3, stride=1, name='layer1_conv1')
    calc.add_conv(3, stride=1, name='layer1_conv2')
    calc.add_conv(3, stride=1, name='layer1_conv3')
    calc.add_conv(3, stride=1, name='layer1_conv4')
    
    # Layer 2 (stride=2)
    calc.add_conv(3, stride=2, name='layer2_conv1')
    calc.add_conv(3, stride=1, name='layer2_conv2')
    calc.add_conv(3, stride=1, name='layer2_conv3')
    calc.add_conv(3, stride=1, name='layer2_conv4')
    
    # Layer 3 (stride=2)
    calc.add_conv(3, stride=2, name='layer3_conv1')
    calc.add_conv(3, stride=1, name='layer3_conv2')
    calc.add_conv(3, stride=1, name='layer3_conv3')
    calc.add_conv(3, stride=1, name='layer3_conv4')
    
    # Layer 4 (stride=2)
    calc.add_conv(3, stride=2, name='layer4_conv1')
    calc.add_conv(3, stride=1, name='layer4_conv2')
    calc.add_conv(3, stride=1, name='layer4_conv3')
    calc.add_conv(3, stride=1, name='layer4_conv4')
    
    return calc.print_summary()

resnet_results = calculate_resnet_rf()

In [ ]:
def visualize_network_rf_comparison():
    """ネットワークの受容野比較"""
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # VGG16のRF成長
    vgg_rfs = [r['rf'] for r in vgg_results]
    vgg_names = [r['name'] for r in vgg_results]
    
    ax = axes[0]
    ax.bar(range(len(vgg_rfs)), vgg_rfs, color='steelblue', alpha=0.7)
    ax.set_xticks(range(len(vgg_rfs)))
    ax.set_xticklabels(vgg_names, rotation=45, ha='right', fontsize=8)
    ax.set_ylabel('受容野サイズ', fontsize=12)
    ax.set_title('VGG16の受容野成長', fontsize=14, fontweight='bold')
    ax.axhline(y=224, color='red', linestyle='--', label='入力サイズ (224)')
    ax.legend()
    
    # ResNet-18のRF成長
    resnet_rfs = [r['rf'] for r in resnet_results]
    resnet_names = [r['name'] for r in resnet_results]
    
    ax = axes[1]
    ax.bar(range(len(resnet_rfs)), resnet_rfs, color='coral', alpha=0.7)
    ax.set_xticks(range(len(resnet_rfs)))
    ax.set_xticklabels(resnet_names, rotation=45, ha='right', fontsize=8)
    ax.set_ylabel('受容野サイズ', fontsize=12)
    ax.set_title('ResNet-18の受容野成長', fontsize=14, fontweight='bold')
    ax.axhline(y=224, color='red', linestyle='--', label='入力サイズ (224)')
    ax.legend()
    
    plt.tight_layout()
    plt.show()
    
    print(f"\nVGG16最終受容野: {vgg_rfs[-1]} (入力224に対して{vgg_rfs[-1]/224*100:.1f}%カバー)")
    print(f"ResNet-18最終受容野: {resnet_rfs[-1]} (入力224に対して{resnet_rfs[-1]/224*100:.1f}%カバー)")

visualize_network_rf_comparison()

### 観察

1. **VGG16**: 最終受容野 = 212 （入力224の約95%）
   - すべて3×3カーネル
   - プーリングで受容野を急速に拡大

2. **ResNet-18**: 最終受容野 = 435 （入力224の194%、つまり入力全体を余裕でカバー）
   - 7×7の初期カーネル
   - ストライド2で効率的に拡大

<a id="summary"></a>
## 7. まとめ

### 学んだこと

1. **受容野の漸化式**
   $$\text{RF}_n = \text{RF}_{n-1} + (k_n - 1) \times \text{累積ストライド}$$

2. **累積ストライドの効果**
   - ストライド>1は受容野成長を加速
   - 入力解像度も低下するトレードオフ

3. **設計原則**
   - 小さいカーネルを重ねる方がパラメータ効率が良い
   - 非線形性も増えて表現力向上
   - ストライドやプーリングで効率的に受容野を拡大

4. **階層的抽象化との関係**
   - 浅い層：局所的特徴（エッジ）
   - 深い層：大域的特徴（オブジェクト）

### 次のノートブック

次のノートブックでは、ダウンサンプリング（プーリングとストライド畳み込み）について詳しく学びます。

- プーリングの種類と特性
- ストライド畳み込み vs プーリング
- 空間解像度と受容野のトレードオフ